## Retail Sales

---

In [1]:
import pandas as pd
import altair as alt
import numpy as np
import re

# Imports for reading FRED API
from os import environ
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

try:
    # for local execution
    apiKeyFromFile = open("/Users/kyledunn/fredApiKey.txt", "r").read().strip()
except FileNotFoundError:
    apiKeyFromFile = None
    pass
# for CI
apiKey = environ.get("FRED_API_KEY", apiKeyFromFile)

def getSeries(series="", apiKey=apiKey, description=None):
    # Construct URL template, fetching as much data as possible
    fetchCommand = "https://api.stlouisfed.org/fred/series/observations?series_id={s}&realtime_end=9999-12-31&api_key={k}&file_type=txt" 
    
    # Call the FRED API
    resp = urlopen(fetchCommand.format(s=series, k=apiKey))
    
    # Read and extract the data from the Zipfile response
    zipfile = ZipFile(BytesIO(resp.read()))
    filesInZip = zipfile.namelist()
    data = zipfile.open(filesInZip[1])
    
    if description is None:
        description = series
    
    # Make a well-formed dataframe
    df = pd.read_csv(data, sep="\t", header=None, skiprows=1,
                       names=["date", description, "rt_start", "rt_end"], na_values=".")
    
    df['date'] = pd.to_datetime(df.date)
    
    return df.set_index("date")

df_cpi = getSeries("CWUR0000SA0")

In [2]:
# Encode the monthly retail sales column names
yearColumns = lambda y : map(lambda s: s.format(y), ['Jan. {}', 'Feb. {}', 'Mar. {}', 'Apr. {}', 'May {}', 'Jun. {}', 'Jul. {}', 'Aug. {}', 'Sep. {}', 'Oct. {}', 'Nov. {}', 'Dec. {}', 'Total {}'])
standardColumns = lambda y : ['NAICS', 'Description'] + list(yearColumns(y))

# Note, need to add to 2019 as new data is released
# TODO, try to automate this better
oneOffYears = {
    '2019': standardColumns('2019')[:2] + ['Jan. 2019', 'Feb. 2019', 'Mar. 2019', 'Apr. 2019', 'May 2019', 'Jun. 2019', 'Jul. 2019', 'Aug. 2019', 'Sep. 2019', 'Oct. 2019', 'Nov. 2019', 'Dec. 2019', '2019 CUM', '2019 PY CUM'],
    '2016': standardColumns('2016') + ['IGNORE']
}

# Fetch the historical monthly retail sales data
# set sheet_name to none to read all sheets in the XLS
dfs = pd.read_excel('https://www.census.gov/retail/mrts/www/mrtssales92-present.xls', sheet_name=None)

# parse the results sheet-by-sheet, normalizing the columns names and index
allDfs = []
for y in dfs.keys():
    #print(dfs[y].columns)
    dfs[y].columns = oneOffYears.get(y) or standardColumns(y)
    dfs[y] = dfs[y].drop('NAICS', axis=1)
    #print(dfs[y].iloc[5:109].set_index('Description').index.is_unique)
    allDfs.append(dfs[y].iloc[5:109].set_index('Description'))

# workaround for merging the sheets into a single dataframe
old = pd.concat(allDfs[2:], axis=1)
new = pd.concat(allDfs[:2], axis=1)
combined = new.join(old).drop('ADJUSTED(2)').T

#combined.head(10)

In [3]:
# subset the combined dataset to the monthly report records
tsNormalized = combined[combined.index.map(lambda v: re.search('[A-z]{3}.? \d{4}$', v) is not None and 'Total' not in v)].copy()

# create a proper datetime column
# note: (p) "preliminary" records lose their designation and are treated the same as final records
tsNormalized['dt'] = tsNormalized.index.map(lambda v: pd.to_datetime(v.replace('(p)', ''), format='%b. %Y') 
                                            if '.' in v else pd.to_datetime(v, format='%b %Y'))

# reindex the dataframe on the datetime field and ensure all values are numeric
dt = tsNormalized.set_index('dt')
dt = dt.apply(pd.to_numeric, errors='coerce')
#dt.head()

# Melt and re-aggregate to combine duplicate categories
df_melted = dt.reset_index().melt(id_vars='dt')
df_agg = df_melted.groupby(['dt', 'Description']).agg(sum).reset_index('Description')
#df_agg.head()

# Reshape the dataframe so each monhtly report is a row and each category is a column
df_combined = df_agg.pivot(index=df_agg.index, columns='Description')
df_combined.columns = df_combined.columns.droplevel()
#df_combined.tail()

### What is the trend for home furnishing retail stores?

In [4]:
def doLineChartFor(metric, df, yLabel='Monthly Revenue [Million USD]', color='purple'):
    return alt.Chart(df.reset_index()).mark_line(color=color).encode(
        alt.X('dt', axis=alt.Axis(title='')),
        alt.Y('{}:Q'.format(metric), axis=alt.Axis(title=yLabel)),
        tooltip=[alt.Tooltip('dt:T'), alt.Tooltip('{}:Q'.format(metric))]
    ).properties(
        title='US: {} - over time'.format(metric),
        height=500,
        width=750
    )
    
doLineChartFor('Furniture, home furn, electronics, and appliance stores', df_combined)

alt.Chart(...)

### What is the *growth* trend for home furnishing retail stores?

In [5]:
def doYoYChartFor(metric, df, color=''):
    yoy = df[[metric]].pct_change(12).apply(lambda v: v*100.).sort_index()

    return alt.Chart(yoy[-160:].reset_index()).mark_bar(size=2.5).encode(
        alt.X('dt:T', axis=alt.Axis(title='')),
        alt.Y('{}:Q'.format(metric), axis=alt.Axis(title='Year over Year Revenue Growth [%]'.format(metric))),
        tooltip=[alt.Tooltip('dt:T'), alt.Tooltip('{}:Q'.format(metric))]
    ).properties(
        title='US: {} - growth over time'.format(metric),
        height=500,
        width=750
    )
    
doYoYChartFor('Furniture, home furn, electronics, and appliance stores', df_combined)

alt.Chart(...)

### What is the trend for retail sales revenue?

In [6]:
doLineChartFor('Retail sales, total', df_combined, 'Monthly Revenue [Million USD]')

alt.Chart(...)

### What is the *growth* trend for retail sales revenue?

In [7]:
doYoYChartFor('Retail sales, total', df_combined)

alt.Chart(...)

### What is the *growth* trend for automotive-related retail sales revenue?

In [8]:
as_yoy = df_combined.copy()

# Compute auto-related sales revenues by ( total - (total - autos) ) => autos
as_yoy['Automotive Related'] = (as_yoy['Retail sales, total'] -\
                   as_yoy['Retail sales, total (excl. motor vehicle and parts dealers)'])

as_df = as_yoy['Automotive Related'].reset_index()

doYoYChartFor('Automotive Related', as_df.set_index('dt'))

alt.Chart(...)

### How do the above trends look when adjusting for inflation (CPI)?

In [9]:
toCompare = [
    'Retail sales, total',
    'Retail sales, total (excl. motor vehicle and parts dealers)',
    'Retail sales and food services excl motor vehicle and parts'
]

tmp = df_combined[['Retail sales, total', 'Retail sales, total (excl. motor vehicle and parts dealers)']].copy()
tmp.columns = ['Retail sales, total', 'Retail sales excluding motor vehicles']

tmp['Retail sales, motor vehicles and parts'] = tmp['Retail sales, total'] - tmp['Retail sales excluding motor vehicles']
tmp = tmp.join(df_cpi['CWUR0000SA0'])
tmp.columns = ['Retail Total', 'Retail Excluding Motor Vehicles', 'Retail Motor Vehicles', 'CPI']

tmp['Total-adj'] = tmp['Retail Total'] / tmp['CPI']
tmp['Auto-adj'] = tmp['Retail Motor Vehicles'] / tmp['CPI']

chartable = tmp.reset_index()[['index', 'Total-adj', 'Auto-adj']].melt(id_vars='index')

alt.Chart(chartable).mark_line().encode(
    alt.X('index:T', axis=alt.Axis(title='')),
    alt.Y('value:Q', axis=alt.Axis(title='Monthly Revenue [Million 1982-1984 USD]')),
    alt.Color('variable:N')
).properties(
    title='US: CPI-adjusted Retail Sales Revenue',
    height=500,
    width=700
)

alt.Chart(...)

### What is the *growth* trend for retail sales when adjusted for inflation (CPI)?

In [10]:
df_adj = tmp['Total-adj'].reset_index().copy()
df_adj.columns = ['dt', 'Retail Sales Adj']

doYoYChartFor('Retail Sales Adj', df_adj.set_index('dt'))

alt.Chart(...)

### What is the trend for non-store retailers (ecommerce)?

In [11]:
doLineChartFor('Nonstore retailers', df_combined)

alt.Chart(...)